In [1]:
#Import libraries

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#Image dimensions 

img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Downloads/Data/train'
validation_data_dir = 'Downloads/Data/validation' 
nb_train_samples = 2000 
nb_validation_samples = 800
epochs = 20
batch_size = 16

In [8]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # Build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,  # Only yield batches of data, not labels
        shuffle=False)    # Keep the order of the data same 
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),  # Save the output as numpy array 
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    #Retrieving the labels on trainset
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb')) 
    #Retreving the labels on validation set 
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model() 

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.7320 - acc: 0.7435 - val_loss: 0.3738 - val_acc: 0.8200
Epoch 2/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3467 - acc: 0.8595 - val_loss: 0.2580 - val_acc: 0.8962
Epoch 3/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3130 - acc: 0.8800 - val_loss: 0.3111 - val_acc: 0.8625
Epoch 4/20
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2597 - acc: 0.8995 - val_loss: 0.6237 - val_acc: 0.7812
Epoch 5/20
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2433 - acc: 0.9115 - val_loss: 0.2647 - val_acc: 0.9062
Epoch 6/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2016 - acc: 0.9220 - val_loss: 0.3609 - val_acc: 0.8888
Epoch 7/20
2000/2000 [==============================] - 4s 2ms/ste